![rmotr](https://i.imgur.com/jiPp4hj.png)
<hr style="margin-bottom: 40px;">

<img src="https://user-images.githubusercontent.com/7065401/39117440-24199c72-46e7-11e8-8ffc-25c6e27e07d4.jpg"
    style="width:300px; float: right; margin: 0 40px 40px 40px;"></img>

# Handling Missing Data with Pandas

pandas borrows all the capabilities from numpy selection + adds a number of convenient methods to handle missing values. Let's see one at a time:

![separator2](https://i.imgur.com/4gX5WFr.png)

## Hands on!

In [1]:
import numpy as np
import pandas as pd

### Pandas utility functions

Similarly to `numpy`, pandas also has a few utility functions to identify and detect null values:

In [2]:
pd.isnull(np.nan)

True

In [3]:
pd.isnull(None)

True

In [4]:
pd.isna(np.nan)

True

In [5]:
pd.isna(None)

True

The opposite ones also exist:

In [6]:
pd.notnull(None)

False

In [7]:
pd.notnull(np.nan)

False

In [8]:
pd.notna(np.nan)

False

In [9]:
pd.notnull(3)

True

These functions also work with Series and `DataFrame`s:

In [10]:
pd.isnull(pd.Series([1, np.nan, 7]))

,0
0,False
1,True
2,False


In [11]:
pd.notnull(pd.Series([1, np.nan, 7]))

,0
0,True
1,False
2,True


In [12]:
pd.isnull(pd.DataFrame({
    'Column A': [1, np.nan, 7],
    'Column B': [np.nan, 2, 3],
    'Column C': [np.nan, 2, np.nan]
}))

,Column A,Column B,Column C
0,False,True,True
1,True,False,False
2,False,False,True


![separator1](https://i.imgur.com/ZUWYTii.png)

### Pandas Operations with Missing Values

Pandas manages missing values more gracefully than numpy. `nan`s will no longer behave as "viruses", and operations will just ignore them completely:

In [13]:
pd.Series([1, 2, np.nan]).count()

2

In [14]:
pd.Series([1, 2, np.nan]).sum()

3.0

In [15]:
pd.Series([2, 2, np.nan]).mean()

2.0

### Filtering missing data

As we saw with numpy, we could combine boolean selection + `pd.isnull` to filter out those `nan`s and null values:

In [16]:
s = pd.Series([1, 2, 3, np.nan, np.nan, 4])

In [17]:
pd.notnull(s)

,0
0,True
1,True
2,True
3,False
4,False
5,True


In [18]:
pd.isnull(s)

,0
0,False
1,False
2,False
3,True
4,True
5,False


In [19]:
pd.notnull(s).sum()

4

In [20]:
pd.isnull(s).sum()

2

In [21]:
s[pd.notnull(s)]

,0
0,1.0
1,2.0
2,3.0
5,4.0


But both `notnull` and `isnull` are also methods of `Series` and `DataFrame`s, so we could use it that way:

In [22]:
s.isnull()

,0
0,False
1,False
2,False
3,True
4,True
5,False


In [23]:
s.notnull()

,0
0,True
1,True
2,True
3,False
4,False
5,True


In [24]:
s[s.notnull()]

,0
0,1.0
1,2.0
2,3.0
5,4.0


![separator1](https://i.imgur.com/ZUWYTii.png)

### Dropping null values

Boolean selection + `notnull()` seems a little bit verbose and repetitive. And as we said before: any repetitive task will probably have a better, more DRY way. In this case, we can use the `dropna` method:

In [25]:
s

,0
0,1.0
1,2.0
2,3.0
3,NaN
4,NaN
5,4.0


In [26]:
s.dropna()

,0
0,1.0
1,2.0
2,3.0
5,4.0


### Dropping null values on DataFrames

You saw how simple it is to drop `na`s with a Series. But with `DataFrame`s, there will be a few more things to consider, because you can't drop single values. You can only drop entire columns or rows. Let's start with a sample `DataFrame`:

In [28]:
df = pd.DataFrame({
    'Column A': [1, np.nan, 30, np.nan],
    'Column B': [2, 8, 31, np.nan],
    'Column C': [np.nan, 9, 32, 100],
    'Column D': [5, 8, 34, 110],
})

In [29]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [30]:
df.shape

(4, 4)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Column A  2 non-null      float64
 1   Column B  3 non-null      float64
 2   Column C  3 non-null      float64
 3   Column D  4 non-null      int64  
dtypes: float64(3), int64(1)
memory usage: 260.0 bytes


In [32]:
df.isnull()

,Column A,Column B,Column C,Column D
0,False,False,True,False
1,True,False,False,False
2,False,False,False,False
3,True,True,False,False


In [33]:
df.isnull().sum()

,0
Column A,2
Column B,1
Column C,1
Column D,0


The default `dropna` behavior will drop all the rows in which _any_ null value is present:

In [34]:
df.dropna()

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


In this case we're dropping **rows**. Rows containing null values are dropped from the DF. You can also use the `axis` parameter to drop columns containing null values:

In [35]:
df.dropna(axis=1)  # axis='columns' also works

,Column D
0,5
1,8
2,34
3,110


In this case, any row or column that contains **at least** one null value will be dropped. Which can be, depending on the case, too extreme. You can control this behavior with the `how` parameter. Can be either `'any'` or `'all'`:

In [36]:
df2 = pd.DataFrame({
    'Column A': [1, np.nan, 30],
    'Column B': [2, np.nan, 31],
    'Column C': [np.nan, np.nan, 100]
})

In [37]:
df2

,Column A,Column B,Column C
0,1.0,2.0,NaN
1,NaN,NaN,NaN
2,30.0,31.0,100.0


In [38]:
df.dropna(how='all')

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [39]:
df.dropna(how='any')  # default behavior

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


You can also use the `thresh` parameter to indicate a _threshold_ (a minimum number) of non-null values for the row/column to be kept:

In [40]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [41]:
df.dropna(thresh=3)

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34


In [42]:
df.dropna(thresh=3, axis='columns')

,Column B,Column C,Column D
0,2.0,NaN,5
1,8.0,9.0,8
2,31.0,32.0,34
3,NaN,100.0,110


![separator1](https://i.imgur.com/ZUWYTii.png)

### Filling null values

Sometimes instead than dropping the null values, we might need to replace them with some other value. This highly depends on your context and the dataset you're currently working. Sometimes a `nan` can be replaced with a `0`, sometimes it can be replaced with the `mean` of the sample, and some other times you can take the closest value. Again, it depends on the context. We'll show you the different methods and mechanisms and you can then apply them to your own problem.

In [43]:
s

,0
0,1.0
1,2.0
2,3.0
3,NaN
4,NaN
5,4.0


**Filling nulls with a arbitrary value**

In [44]:
s.fillna(0)

,0
0,1.0
1,2.0
2,3.0
3,0.0
4,0.0
5,4.0


In [45]:
s.fillna(s.mean())

,0
0,1.0
1,2.0
2,3.0
3,2.5
4,2.5
5,4.0


In [46]:
s

,0
0,1.0
1,2.0
2,3.0
3,NaN
4,NaN
5,4.0


**Filling nulls with contiguous (close) values**

The `method` argument is used to fill null values with other values close to that null one:

In [47]:
s.fillna(method='ffill')

<ipython-input-47-1b2d630da9dd>:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s.fillna(method='ffill')


,0
0,1.0
1,2.0
2,3.0
3,3.0
4,3.0
5,4.0


In [48]:
s.fillna(method='bfill')

<ipython-input-48-3624d7f4ebdb>:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s.fillna(method='bfill')


,0
0,1.0
1,2.0
2,3.0
3,4.0
4,4.0
5,4.0


This can still leave null values at the extremes of the Series/DataFrame:

In [49]:
pd.Series([np.nan, 3, np.nan, 9]).fillna(method='ffill')

<ipython-input-49-80c03cda8078>:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pd.Series([np.nan, 3, np.nan, 9]).fillna(method='ffill')


,0
0,NaN
1,3.0
2,3.0
3,9.0


In [50]:
pd.Series([1, np.nan, 3, np.nan, np.nan]).fillna(method='bfill')

<ipython-input-50-7dcc4cf3c0e4>:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pd.Series([1, np.nan, 3, np.nan, np.nan]).fillna(method='bfill')


,0
0,1.0
1,3.0
2,3.0
3,NaN
4,NaN


### Filling null values on DataFrames

The `fillna` method also works on `DataFrame`s, and it works similarly. The main differences are that you can specify the `axis` (as usual, rows or columns) to use to fill the values (specially for methods) and that you have more control on the values passed:

In [51]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [52]:
df.fillna({'Column A': 0, 'Column B': 99, 'Column C': df['Column C'].mean()})

,Column A,Column B,Column C,Column D
0,1.0,2.0,47.0,5
1,0.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,0.0,99.0,100.0,110


In [53]:
df.fillna(method='ffill', axis=0)

<ipython-input-53-8dfdc675c751>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', axis=0)


,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,1.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,30.0,31.0,100.0,110


In [54]:
df.fillna(method='ffill', axis=1)

<ipython-input-54-ffb0f85e8bb9>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', axis=1)


,Column A,Column B,Column C,Column D
0,1.0,2.0,2.0,5.0
1,NaN,8.0,9.0,8.0
2,30.0,31.0,32.0,34.0
3,NaN,NaN,100.0,110.0


![separator1](https://i.imgur.com/ZUWYTii.png)

### Checking if there are NAs

The question is: Does this `Series` or `DataFrame` contain any missing value? The answer should be yes or no: `True` or `False`. How can you verify it?

**Example 1: Checking the length**

If there are missing values, `s.dropna()` will have less elements than `s`:

In [55]:
s.dropna().count()

4

In [56]:
missing_values = len(s.dropna()) != len(s)
missing_values

True

There's also a `count` method, that excludes `nan`s from its result:

In [57]:
len(s)

6

In [58]:
s.count()

4

So we could just do:

In [59]:
missing_values = s.count() != len(s)
missing_values

True

**More Pythonic solution `any`**

The methods `any` and `all` check if either there's `any` True value in a Series or `all` the values are `True`. They work in the same way as in Python:

In [60]:
pd.Series([True, False, False]).any()

True

In [61]:
pd.Series([True, False, False]).all()

False

In [62]:
pd.Series([True, True, True]).all()

True

The `isnull()` method returned a Boolean `Series` with `True` values wherever there was a `nan`:

In [63]:
s.isnull()

,0
0,False
1,False
2,False
3,True
4,True
5,False


So we can just use the `any` method with the boolean array returned:

In [64]:
pd.Series([1, np.nan]).isnull().any()

True

In [65]:
pd.Series([1, 2]).isnull().any()

False

In [66]:
s.isnull().any()

True

A more strict version would check only the `values` of the Series:

In [67]:
s.isnull().values

array([False, False, False,  True,  True, False])

In [68]:
s.isnull().values.any()

True

![separator2](https://i.imgur.com/4gX5WFr.png)